In [2]:
import cv2
from deepface import DeepFace
import datetime
import time

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

# Dictionary to track unique emotions
logged_emotions = set()

# Timer to manage 10-second intervals
last_logged_time = time.time()

print("Emotion detection started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert the frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Crop the face region for emotion analysis
        face_roi = frame[y:y + h, x:x + w]

        try:
            # Analyze the face for emotions
            analysis = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
            dominant_emotion = analysis[0]['dominant_emotion']

            # Log emotion every 10 seconds without repetition
            current_time = time.time()
            if current_time - last_logged_time >= 10 and dominant_emotion not in logged_emotions:
                # Display the dominant emotion on the video feed
                cv2.putText(frame, f"{dominant_emotion}", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2, cv2.LINE_AA)

                # Log the emotion to the text file with a timestamp (real-time)
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                log_entry = f"{timestamp} - Detected Emotion: {dominant_emotion}\n"
                with open("emotion_report.txt", "a") as log_file:
                    log_file.write(log_entry)
                print(f"Logged emotion: {dominant_emotion}")

                # Add the emotion to the set
                logged_emotions.add(dominant_emotion)

                # Update the last logged time
                last_logged_time = current_time

        except Exception as e:
            print(f"Error analyzing emotions: {e}")

    # Show the live feed with face bounding boxes and emotion annotations
    cv2.imshow("Face Detection and Emotion Recognition", frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Finalize the report in the text file
with open("emotion_report.txt", "a") as log_file:
    log_file.write("\nSession ended at: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")

# Release resources
cap.release()
cv2.destroyAllWindows()

print("Emotion detection session ended. Check 'emotion_report.txt' for the results.")


Emotion detection started. Press 'q' to quit.
Logged emotion: surprise
Logged emotion: angry
Logged emotion: neutral
Logged emotion: sad
Logged emotion: happy
Logged emotion: fear
Emotion detection session ended. Check 'emotion_report.txt' for the results.
